In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT
        (SELECT COUNT()
        FROM boston
        WHERE CRIM IS NULL) AS CRIM_Null,

        (SELECT COUNT()
        FROM boston
        WHERE ZN IS NULL) AS ZN_Null,

        (SELECT COUNT()
        FROM boston
        WHERE INDUS IS NULL) AS INDUS_Null,

        (SELECT COUNT()
        FROM boston
        WHERE CHAS IS NULL) AS CHAS_Null,
        
        (SELECT COUNT()
        FROM boston
        WHERE NOX IS NULL) AS NOX_Null  
    FROM boston
    LIMIT 1

    ''',
    con,
)

,CRIM_Null,ZN_Null,INDUS_Null,CHAS_Null,NOX_Null
0,0,0,0,0,0


In [8]:
# так как значений NULL нет, поищем нулевые значения
pd.read_sql(
    '''
    SELECT
        (SELECT COUNT()
        FROM boston
        WHERE CRIM = '0.0') AS CRIM_ZERO,

        (SELECT COUNT()
        FROM boston
        WHERE ZN = '0.0') AS ZNI_ZERO,

        (SELECT COUNT()
        FROM boston
        WHERE INDUS = '0.0') AS INDUS_ZERO,

        (SELECT COUNT()
        FROM boston
        WHERE CHAS = '0.0') AS CHAS_ZERO,
        
        (SELECT COUNT()
        FROM boston
        WHERE NOX = '0.0') AS NOX_ZERO
    FROM boston
    LIMIT 1

    ''',
    con,
)

,CRIM_ZERO,ZNI_ZERO,INDUS_ZERO,CHAS_ZERO,NOX_ZERO
0,0,372,0,471,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
        COUNT(DISTINCT CRIM) as UniqueCRIM, 
        COUNT(DISTINCT ZN) as UniqueZN, 
        COUNT(DISTINCT INDUS) as UniqueINDUS, 
        COUNT(DISTINCT CHAS) as UniqueCHAS, 
        COUNT(DISTINCT NOX) as UniqueNOX,
        COUNT(DISTINCT LSTAT) as UniqueLSTAT
    FROM boston
    ''',
    con,
)

,UniqueCRIM,UniqueZN,UniqueINDUS,UniqueCHAS,UniqueNOX,UniqueLSTAT
0,504,26,76,2,81,455


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [10]:
tasks+=1
pd.read_sql(
    '''
    SELECT Name 
    FROM (
        SELECT
            (SELECT 'CRIM') AS Name, 
            (SELECT AVG(CRIM)
                FROM (
                        SELECT CRIM
                        FROM boston
                        ORDER BY CRIM
                        LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
                        OFFSET (SELECT (COUNT(*) - 1) / 2
                                FROM boston))) AS median, 
            (SELECT MIN(CRIM) 
            FROM boston) AS min
        FROM boston
        UNION
        SELECT
            (SELECT 'ZN') AS Name, 
            (SELECT AVG(ZN)
                FROM (
                        SELECT ZN
                        FROM boston
                        ORDER BY ZN
                        LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
                        OFFSET (SELECT (COUNT(*) - 1) / 2
                                FROM boston))) AS median, 
            (SELECT MIN(ZN) FROM boston) AS min
        FROM boston
        UNION
        SELECT
            (SELECT 'INDUS') AS Name, 
            (SELECT AVG(INDUS)
                FROM (
                        SELECT INDUS
                        FROM boston
                        ORDER BY INDUS
                        LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
                        OFFSET (SELECT (COUNT(*) - 1) / 2
                                FROM boston))) AS median, 
            (SELECT MIN(INDUS) FROM boston) AS min
        FROM boston
        UNION
        SELECT
            (SELECT 'CHAS') AS Name, 
            (SELECT AVG(CHAS)
                FROM (
                        SELECT CHAS
                        FROM boston
                        ORDER BY CHAS
                        LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
                        OFFSET (SELECT (COUNT(*) - 1) / 2
                                FROM boston))) AS median, 
            (SELECT MIN(CHAS) FROM boston) AS min
        FROM boston
        UNION
        SELECT
            (SELECT 'NOX') AS Name, 
            (SELECT AVG(NOX)
                FROM (
                        SELECT NOX
                        FROM boston
                        ORDER BY NOX
                        LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
                        OFFSET (SELECT (COUNT(*) - 1) / 2
                                FROM boston))) AS median, 
            (SELECT MIN(NOX) FROM boston) AS min
        FROM boston)
    WHERE median = min
    ''',
    con,
)

,Name
0,CHAS
1,ZN


Вывод: половина или больше половины домов:
- расположены не у реки (CHAS = '0.0')
- не имеют долю земли под жилую застройку, зонированную под участки площадью более 25 000 кв. футов (ZN = '0.0')

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [11]:
tasks+=1
pd.read_sql(
    '''
    WITH top_25 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV DESC
        limit 25),
        low_25 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV
        limit 25),

        top_50 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV DESC
        limit 50),
        low_50 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV
        limit 50),

        top_100 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV DESC
        limit 100),
        low_100 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV
        limit 100),

        top_200 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV DESC
        limit 200),
        low_200 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV
        limit 200),

        top_300 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV DESC
        limit 300),
        low_300 AS
        (SELECT RM
        FROM boston
        ORDER BY MEDV
        limit 300)

    SELECT
        (SELECT 25) AS Q,
        (SELECT AVG(RM) FROM top_25) AS top,
        (SELECT AVG(RM) FROM low_25) AS low,
        (SELECT AVG(RM) FROM top_25) - (SELECT AVG(RM) FROM low_25) AS dif
    FROM boston
    UNION
    SELECT
        (SELECT 50) AS Q,
        (SELECT AVG(RM) FROM top_50) AS top,
        (SELECT AVG(RM) FROM low_50) AS low,
        (SELECT AVG(RM) FROM top_50) - (SELECT AVG(RM) FROM low_50) AS dif
    FROM boston
    UNION
    SELECT
        (SELECT 100) AS Q,
        (SELECT AVG(RM) FROM top_100) AS top,
        (SELECT AVG(RM) FROM low_100) AS low,
        (SELECT AVG(RM) FROM top_100) - (SELECT AVG(RM) FROM low_100) AS dif
    FROM boston
    UNION
    SELECT
        (SELECT 200) AS Q,
        (SELECT AVG(RM) FROM top_200) AS top,
        (SELECT AVG(RM) FROM low_200) AS low,
        (SELECT AVG(RM) FROM top_200) - (SELECT AVG(RM) FROM low_200) AS dif
    FROM boston
    UNION
    SELECT
        (SELECT 300) AS Q,
        (SELECT AVG(RM) FROM top_300) AS top,
        (SELECT AVG(RM) FROM low_300) AS low,
        (SELECT AVG(RM) FROM top_300) - (SELECT AVG(RM) FROM low_300) AS dif
    FROM boston
    ''',
    con,
)

,Q,top,low,dif
0,25,7.637320,5.747840,1.889480
1,50,7.487060,5.753240,1.733820
2,100,7.198760,5.887120,1.311640
3,200,6.758680,5.911705,0.846975
4,300,6.539007,5.972227,0.566780


Вывод: чем больше количество комнат в доме, тем дороже стоит дом. Количество комнат влияет на стоимость дома сильно.


5.  Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [12]:
tasks+=1
pd.read_sql(
    '''
    WITH rank_LSTAT AS (
        SELECT LSTAT, 
        MEDV, 
        rank() OVER (PARTITION BY LSTAT ORDER BY MEDV) rankLSTAT
        FROM boston)

    SELECT LSTAT, MEDV, rankLSTAT
    FROM rank_LSTAT
    WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
    ORDER BY rankLSTAT
    ''',
    con,
)

,LSTAT,MEDV,rankLSTAT
0,1.73,50.0,1
1,1.92,50.0,1
2,2.88,50.0,1
3,2.96,50.0,1
4,2.97,50.0,1
5,3.26,50.0,1
6,3.32,50.0,1
7,3.70,50.0,1
8,3.73,50.0,1
9,4.63,50.0,1


Вывод: прослеживается зависимость, что чем ниже процент населения с низким доходом (LSTAT), тем выше стоимость дома(MEDV). 

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT ROUND(AVG(MEDV), 2) AS AVG_cost, CHAS
    FROM boston
    GROUP BY CHAS
    ''',
    con,
)

,AVG_cost,CHAS
0,22.09,0.0
1,28.44,1.0


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT a.NAME
    FROM(
        SELECT 
            (SELECT 'CRIM') AS NAME,
            (SELECT AVG(CRIM) FROM boston where CHAS = 1) - (SELECT AVG(CRIM) FROM boston where CHAS = 0) AS dif
        FROM boston
        UNION
        SELECT 
            (SELECT 'ZN') AS NAME,
            (SELECT AVG(ZN) FROM boston where CHAS = 1) - (SELECT AVG(ZN) FROM boston where CHAS = 0) AS dif
        FROM boston
        UNION
        SELECT 
            (SELECT 'INDUS') AS NAME,
            (SELECT AVG(INDUS) FROM boston where CHAS = 1) - (SELECT AVG(INDUS) FROM boston where CHAS = 0) AS dif
        FROM boston
        UNION
        SELECT 
            (SELECT 'CHAS') AS NAME,
            (SELECT AVG(CHAS) FROM boston where CHAS = 1) - (SELECT AVG(CHAS) FROM boston where CHAS = 0) AS dif
        FROM boston
        UNION
        SELECT 
            (SELECT 'NOX') AS NAME,
            (SELECT AVG(NOX) FROM boston where CHAS = 1) - (SELECT AVG(NOX) FROM boston where CHAS = 0) AS dif
        FROM boston) a
    WHERE dif > 0
    ''',
    con,
)

,NAME
0,CHAS
1,INDUS
2,NOX


Вывод: если дом граничит с рекой, то
- выше доля промышленной застройки (INDUS)
- выше концентрация оксидов азота (NOX)

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [15]:
tasks+=1
pd.read_sql(
    '''
    SELECT PERCENTILE, round(AVG(INDUS), 2) AS INDUS, round(AVG(NOX), 2) AS NOX
    FROM
        (SELECT INDUS, NOX,
            (NTILE(10) OVER (ORDER BY INDUS, NOX))*10 AS 'PERCENTILE'
        FROM boston)
    GROUP BY PERCENTILE
    ''',
    con,
)

,PERCENTILE,INDUS,NOX
0,10,2.02,0.44
1,20,3.65,0.49
2,30,5.25,0.45
3,40,6.47,0.47
4,50,8.26,0.53
5,60,10.75,0.52
6,70,17.25,0.56
7,80,18.10,0.68
8,90,18.31,0.71
9,100,21.94,0.69


Вывод: чем выше доля промышленной застройки, тем выше концентрация оксидов азота

In [16]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
